In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import sys
import os
import io
import shutil

import pudl
import pudl.glue.ferc1_eia
from pudl import init
from pudl import constants as pc
import tableschema
import hashlib
import datapackage
import json
import logging
from tableschema import Table
from tableschema import Schema
import datetime
import goodtables

In [ ]:
from pudl import ETL_pkg
from pudl.ETL_pkg import ETL_pkg, _input_validate
from pudl.ETL_pkg import _ETL_eia_pkg, _ETL_ferc1_pkg, _ETL_glue

In [8]:
pudl_settings = pudl.settings.init()

In [16]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [4]:
from sqlalchemy.engine import reflection
pudl_engine = init.connect_db(testing=False)
insp = reflection.Inspector.from_engine(pudl_engine)
# extract the table names
tbls = insp.get_table_names()
# extract only the ferc tables
ferc_tbls = [s for s in tbls if "ferc" in s]

If you want to generate the package for just ferc as an example, you can use the commented out ferc options below.

In [5]:
name = "pudl-test"
title = "All tables integrated into PUDL."
tbls = tbls

#name = "pudl-ferc1"
#title =  "FERC Form 1 tables integrated into PUDL."
#tbls = ferc_tbls

In [ ]:
#tbls.remove('hourly_emissions_epacems')

In [6]:
# we need this as the main info regarding this iteration of packaging
pkg_skeleton = {
    "name": name,
    "title": title,
    "description": "A full metadat description of all PUDL tables.",
    "keywords": [
        "ferc",
        "form 1",
        "energy",
        "electricity",
        "utility",
        "fuel",
        "expenses",
        "coal",
        "natural gas",
        "generation",
        "regulation"
    ]
}

this will generate metadata and csv's for every table in pudl. they will live in `results/data_pkgs`

In [11]:
pudl_settings

{'pudl_in': '/Users/christinagosnell/code/pudl',
 'pudl_out': '/Users/christinagosnell/code/pudl/results',
 'data_dir': '/Users/christinagosnell/code/pudl/data',
 'settings_dir': '/Users/christinagosnell/code/pudl/settings',
 'notebook_dir': '/Users/christinagosnell/code/pudl/results/notebooks',
 'sqlite_dir': '/Users/christinagosnell/code/pudl/results/sqlite',
 'parquet_dir': '/Users/christinagosnell/code/pudl/results/parquet',
 'datapackage_dir': '/Users/christinagosnell/code/pudl/results/datapackage',
 'db_pudl': {'drivername': 'postgresql',
  'host': '127.0.0.1',
  'username': 'catalyst',
  'database': 'pudl'},
 'db_pudl_test': {'drivername': 'postgresql',
  'host': '127.0.0.1',
  'username': 'catalyst',
  'database': 'pudl_test'},
 'ferc1_sqlite_url': 'sqlite:////Users/christinagosnell/code/pudl/results/sqlite/ferc1.sqlite',
 'ferc1_test_sqlite_url': 'sqlite:////Users/christinagosnell/code/pudl/results/sqlite/ferc1_test.sqlite'}

In [22]:
pkg = pudl.output.export.data_package(tbls,
                                      pkg_skeleton,
                                      pudl_settings, 
                                      dry_run=True)

Finding dependent tables for plants_ferc
Finding dependent tables for utilities_ferc
Finding dependent tables for utilities
Finding dependent tables for plants
Finding dependent tables for plants_ferc
Finding dependent tables for utilities_ferc
Finding dependent tables for utilities
Finding dependent tables for plants
Finding dependent tables for utilities_ferc
Finding dependent tables for utilities
Finding dependent tables for plants
Finding dependent tables for plants_ferc
Finding dependent tables for utilities_ferc
Finding dependent tables for utilities
Finding dependent tables for plants
Finding dependent tables for plants
Finding dependent tables for plants_entity_eia
Finding dependent tables for plants_entity_eia
Finding dependent tables for utilities
Finding dependent tables for utilities_entity_eia
Finding dependent tables for utilities
Finding dependent tables for plants
Finding dependent tables for utilities
Finding dependent tables for plants_entity_eia
Finding dependent tab

Skipping export of plants_steam_ferc1 to /Users/christinagosnell/code/pudl/package_data/meta/datapackage/pudl-test/data/plants_steam_ferc1.csv
Skipping export of utilities_eia860 to /Users/christinagosnell/code/pudl/package_data/meta/datapackage/pudl-test/data/utilities_eia860.csv
Skipping export of boilers_entity_eia to /Users/christinagosnell/code/pudl/package_data/meta/datapackage/pudl-test/data/boilers_entity_eia.csv
Skipping export of plants_entity_eia to /Users/christinagosnell/code/pudl/package_data/meta/datapackage/pudl-test/data/plants_entity_eia.csv
Skipping export of transmission_joint_ipm to /Users/christinagosnell/code/pudl/package_data/meta/datapackage/pudl-test/data/transmission_joint_ipm.csv
Finding dependent tables for plants_ferc
Finding dependent tables for utilities_ferc
Finding dependent tables for utilities
Finding dependent tables for plants
Finding dependent tables for regions_entity_ipm
Finding dependent tables for regions_entity_ipm
Finding dependent tables fo

In [ ]:
out_dir = SETTINGS['out_dir']
settings_init = pudl.settings.settings_init(settings_file='settings_init_pudl_package.yml')

In [ ]:
metas = pudl.output.export.generate_data_packages(settings_init, debug=True)

- Cems_pkg
    - data
        - cems2017.csv.zip
        - cems2016.csv.zip
        - cems2015.csv.zip
        (see "Compression of Resources": http://frictionlessdata.io/specs/patterns/)
    - datapackage.json
        which includes a list of paths in "paths" in the resource
        (see "Data in Multiple Files": https://frictionlessdata.io/specs/data-resource/)


I need to convert the ETL cems process to generate years of cems at a time and write it compressed.

In [2]:
pkg_settings = pudl.package_data.settings.settings_init(os.path.join(pudl_settings['pudl_in'],'scripts','settings_init_pudl_package.yml'))

AttributeError: module 'pudl.package_data' has no attribute 'settings'

In [ ]:
metas = pudl.output.export.generate_data_packages(pkg_settings, pudl_settings, debug=True)

In [ ]:
#ferc_vali = metas['ferc1_pkg'][1]
#ferc_meta = metas['ferc1-pkg'][0]
#eia_vali = metas['eia_pkg']
cems_vali = metas['epacems_eia860_pkg']

In [ ]:
cems_vali[1]

In [ ]:
glue_vali = metas['glue_pkg']

In [ ]:
glue_pkg = glue_vali[0]

In [ ]:
glue_pkg.attributes

In [ ]:
metas.keys()

In [ ]:
cems_vali = metas['epacems_eia860_pkg']

In [ ]:
cems_vali

In [ ]:
pc.tab_map_eia923.columns

In [ ]:
pc.pudl_tables['eia923']

- the main coordinating function is pudl.output.export.generate_data_packages()
    - uses settings:
        -yml file (ex: ‘settings_init_pudl_package.yml’) and settings.py
    - validates settings using `pudl.ETL_pkg._input_validate`... this spits out a new, validated list of package settings
    - then for each of the of the packages definied in settings, run the ETL_pkg function which runs a data source specific function for each dataset. The ETL_pgk function needs the settings/inputs. In a stand-alone version of this function these can be validated settings or non-validated settings. The settings will get re-validated immediately inside of the data source specific ETL function. These ETL functions generate CSVs in the out_dir/(package name)/data folder.
    - an output of the ETL_pkg function is a list of tables being generated. this list is piped through to a `test_file_consistency` function, which ensures the ETL tables, the CSVs and dependencies from the metadata
    - then the `generate_metadata` function is run. at a high level, this generates and validates the `datapackage.json` file and runs `goodtables.validate`.